In [1]:
## CSV Files downloaded from https://www.data.gouv.fr/fr/datasets/repertoire-national-des-associations/  Fichier RNA Waldec du 01 Mars 2022
## https://towardsdatascience.com/how-to-use-pyspark-on-your-computer-9c7180075617

import os
os.environ['SPARK_HOME']="/opt/spark-3.2.1"
os.environ['PYSPARK_DRIVER_PYTHON']="jupyter"
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']="jupyter"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext(appName="List of associations")

22/05/25 15:48:56 WARN Utils: Your hostname, DESKTOP-3481LPC resolves to a loopback address: 127.0.1.1; using 172.17.3.146 instead (on interface eth0)
22/05/25 15:48:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/25 15:48:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("how to read csv file") \
    .getOrCreate()

file_location = "rna_waldec_20220301/"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ";"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("encoding", "ISO-8859-1") \
  .load(file_location)

df.show(5)
df.count()

22/05/25 15:03:15 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+------+-----+------+-------+----------+----------+----------+----------+------+----------+--------------------+--------------------+--------------------+-------------+-------------+---------------+------------+---------------+-------------+----------------+------------+--------------+---------------+--------------------+--------------+--------------+---------------+--------------------+------------+---------------+--------------------+---------+------------+-------+--------+--------------------+--------+-------------------+
|        id| id_ex|siret|rup_mi|gestion|date_creat|date_decla|date_publi|date_disso|nature|groupement|               titre|         titre_court|               objet|objet_social1|objet_social2|adrs_complement|adrs_numvoie|adrs_repetition|adrs_typevoie|    adrs_libvoie|adrs_distrib|adrs_codeinsee|adrs_codepostal|     adrs_libcommune|adrg_declarant|adrg_complemid|adrg_complemgeo|        adrg_libvoie|adrg_distrib|adrg_codepostal|       adrg_achemine|adrg_pa

2119460

In [5]:
df.columns

['id',
 'id_ex',
 'siret',
 'rup_mi',
 'gestion',
 'date_creat',
 'date_decla',
 'date_publi',
 'date_disso',
 'nature',
 'groupement',
 'titre',
 'titre_court',
 'objet',
 'objet_social1',
 'objet_social2',
 'adrs_complement',
 'adrs_numvoie',
 'adrs_repetition',
 'adrs_typevoie',
 'adrs_libvoie',
 'adrs_distrib',
 'adrs_codeinsee',
 'adrs_codepostal',
 'adrs_libcommune',
 'adrg_declarant',
 'adrg_complemid',
 'adrg_complemgeo',
 'adrg_libvoie',
 'adrg_distrib',
 'adrg_codepostal',
 'adrg_achemine',
 'adrg_pays',
 'dir_civilite',
 'siteweb',
 'publiweb',
 'observation',
 'position',
 'maj_time']

In [6]:
from pyspark.sql.functions import col,lower,upper,lit,concat,when,concat_ws
from pyspark import StorageLevel

# S/D pour enlever les associations Dissoutes ou supprimé
# use concat_ws instead of concat to avoid value being null if one is null (concat)
df2 = df.filter(df.titre.contains("CAMEROUN") | df.objet.contains("cameroun") ) \
        .filter(~df.position.isin(["D","S"]) ) \
        .withColumn("titre",upper(col("titre")))\
        .withColumn("objet",lower(col("objet")))\
        .select("id","titre", "objet", "objet_social1","objet_social2","adrs_numvoie","adrs_typevoie","adrs_libvoie","adrs_codepostal","adrs_libcommune", "siteweb") \
        .withColumn("adrs",concat_ws(" ",col("adrs_numvoie"), col("adrs_typevoie"),col("adrs_libvoie"),col('adrs_codepostal'),col('adrs_libcommune'))) \
        .cache()

df2.show()
df2.count()


+----------+--------------------+--------------------+-------------+-------------+------------+-------------+--------------------+---------------+------------------+--------------------+--------------------+
|        id|               titre|               objet|objet_social1|objet_social2|adrs_numvoie|adrs_typevoie|        adrs_libvoie|adrs_codepostal|   adrs_libcommune|             siteweb|                adrs|
+----------+--------------------+--------------------+-------------+-------------+------------+-------------+--------------------+---------------+------------------+--------------------+--------------------+
|W751075369|ASSOCIATION NDOG-...|créer et favorise...|       014000|       000000|          94|          RUE|    des Chataigniers|          95100|        Argenteuil|                null|94 RUE des Chatai...|
|W751101199|CERCLE D'ETUDES D...|étudier les tradi...|       020015|       000000|         148|           AV|            d'Italie|          75013|             Paris|   

1071

In [7]:
from pyspark.sql.functions import length
df2.where(length(col("objet_social1")) >=7).show()
df2.where(length(col("objet_social1")) >=7).count()

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+------------+---------------+---------------+-------+--------------------+
|        id|               titre|               objet|       objet_social1|       objet_social2|        adrs_numvoie|adrs_typevoie|adrs_libvoie|adrs_codepostal|adrs_libcommune|siteweb|                adrs|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+------------+---------------+---------------+-------+--------------------+
|W751224442|ARTS & CULTURE PA...|promotion et déve...| promotion des ar...| organiser des re...| à moyen terme, l...|       006100|        null|            RUE|         Capron|  75018| à moyen terme, l...|
|W751228313|         PARIS-KRIBI|"association dite...| de permettre aux...| de former les je...| sensibiliser les...|       020015|      000000|           null|            RUE|

6

In [8]:
df2.select("id","adrs_codepostal").where(length(col("adrs_codepostal")) < 5).show()

+----------+---------------+
|        id|adrs_codepostal|
+----------+---------------+
|W751224442|            RUE|
|W332023722|              _|
|W763006523|              _|
|W913002330|            RUE|
+----------+---------------+



In [9]:
import requests
import requests_cache
requests_cache.install_cache('demo_cache')

from tqdm import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

def get_info(rna):

    url = "https://entreprise.data.gouv.fr/api/rna/v1/id/"
    r = requests.get(f"{url}/{rna}")
    association = r.json()["association"]
    info = { "objet_social1": association["objet_social1"],
             "objet_social2": association["objet_social2"] ,
             "adrs": "{} {} {}".format(association["adresse_gestion_libelle_voie"],
                                       association["adresse_code_postal"],
                                       association["adresse_libelle_commune"] ),
             "code_postal" : association["adresse_code_postal"] }

    return info

def get_dept_region(code_postal):
    url = "http://api.geonames.org/postalCodeLookupJSON"
    r = requests.get(f"{url}?postalcode={code_postal}&username=mikera&country=FR")
    info = { "dept" :  r.json()["postalcodes"][0].get("adminName2","") ,
             "region": r.json()["postalcodes"][0].get("adminName1","")
    }
    return  info

df3 = df2.toPandas()

# get_info("W751224442")
df3.loc[df3.adrs_codepostal.str.len() < 5 , "adrs_codepostal"] = df3[df3.adrs_codepostal.str.len() < 5].parallel_apply(lambda row: get_info(row["id"])["code_postal"],axis=1)
df3["dept"] = df3.parallel_apply(lambda row: get_dept_region(row["adrs_codepostal"])["dept"],axis=1)
df3["region"] = df3.parallel_apply(lambda row: get_dept_region(row["adrs_codepostal"])["region"],axis=1)

df3.loc[df3.objet_social1.str.len() >=7 , "adrs"] = df3[df3.objet_social1.str.len() >=7].parallel_apply(lambda row: get_info(row["id"])["adrs"],axis=1)
df3.loc[df3.objet_social1.str.len() >=7 , "objet_social2"] = df3[df3.objet_social1.str.len() >=7].parallel_apply(lambda row: get_info(row["id"])["objet_social2"],axis=1)
df3.loc[df3.objet_social1.str.len() >=7 , "objet_social1"] = df3[df3.objet_social1.str.len() >=7].parallel_apply(lambda row: get_info(row["id"])["objet_social1"],axis=1)
df3

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


,id,titre,objet,objet_social1,objet_social2,adrs_numvoie,adrs_typevoie,adrs_libvoie,adrs_codepostal,adrs_libcommune,siteweb,adrs,dept,region
0,W751075369,ASSOCIATION NDOG-NÉM FRANCE ANNF,créer et favoriser les contacts le dialogue dv...,014000,000000,94,RUE,des Chataigniers,95100,Argenteuil,None,94 RUE des Chataigniers 95100 Argenteuil,Val-d'Oise,Île-de-France
1,W751101199,CERCLE D'ETUDES DES TRADITIONS ET DE LA CULTUR...,étudier les traditions et la culture camerouna...,020015,000000,148,AV,d'Italie,75013,Paris,None,148 AV d'Italie 75013 Paris,Paris,Île-de-France
2,W751105303,LES AMIS DE L'INSTITUT CATHOLIQUE DE YAOUNDE C...,contribuer a la réalisation des objectifs de l...,040000,000000,21,RUE,d'Assas,75006,Paris,None,21 RUE d'Assas 75006 Paris,Paris,Île-de-France
3,W751109411,ASSOCIATION DES CHRETIENS DU CAMEROUN POUR LE ...,"réunir, en france comme à l'étranger, au camer...",014035,000000,4,RUE,de Castellane,75008,Paris,www.accd-cameroun.org,4 RUE de Castellane 75008 Paris,Paris,Île-de-France
4,W751118437,ASSOCIATION CAMEROUNAISE D'ENTRAIDE DES RESSOR...,être le lieu de rencontre de tous les boh nset...,019000,000000,8,RUE,Raymond pitet,75017,Paris,None,8 RUE Raymond pitet 75017 Paris,Paris,Île-de-France
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,W051001109,ASSOCIATION DES AMIS DU COLLEGE EVANGELIQUE DE...,soutenir le collège evangélique de bangwa (cam...,020015,000000,17,VILL1,Curial,75019,Paris,None,17 VILL1 Curial 75019 Paris,Paris,Île-de-France
1067,W052001700,AFRICOEUR,faciliter et promouvoir l'entraide et la solid...,009050,020020,None,CHEM1,de Pré Lacour,05140,Aspremont,None,CHEM1 de Pré Lacour 05140 Aspremont,Hautes-Alpes,Provence-Alpes-Côte d'Azur
1068,W2A4000401,ASSOCIATION DE SOUTIEN AUX ENFANTS CAMEROUNAIS...,association à but humanitaire. elle apporte ai...,020000,020020,None,None,Trinité,20137,Porto-Vecchio,None,Trinité 20137 Porto-Vecchio,Corse-du-Sud,Corse
1069,W9T1002080,ASSOCIATIONS DES CAMEROUNAIS DE MAYOTTE (ACAM),rencontre des camerounais résidents ou de pass...,014040,000000,8,RTE,de Vahibé,97600,Mamoudzou,None,8 RTE de Vahibé 97600 Mamoudzou,,Koungou


In [10]:
# df3[df3.dept == ""]
# https://foad-mooc.auf.org/IMG/pdf/2-codes_nationalites-4.pdf
# filter DOM/TOM
display(df3.loc[ (df3.adrs_codepostal.str.startswith("97") == True) | (df3.adrs_codepostal.str.startswith("98") == True)])
df3.loc[df3.id == "W9T1002080","region"]= "Mayotte"
df3.loc[df3.id == "W9T1002080", "dept"] = "Mayotte"

,id,titre,objet,objet_social1,objet_social2,adrs_numvoie,adrs_typevoie,adrs_libvoie,adrs_codepostal,adrs_libcommune,siteweb,adrs,dept,region
759,W9R1005550,KAMER OCEAN INDIEN,"promouvoir l'amitié, la solidarité et la frate...",014040,007075,6 B,CHEM1,Romely,97419,La Possession,None,6 B CHEM1 Romely 97419 La Possession,Réunion,Réunion
1061,W9C1003716,KAMER973-ASSOCIATION DES CAMEROUNAIS DE LA GUYANE,permettre aux personnes d'origine camerounaise...,014040,000000,71,RUE,Lieutenant Goinet,97300,Cayenne,None,71 RUE Lieutenant Goinet 97300 Cayenne,Guyane,Guyane
1069,W9T1002080,ASSOCIATIONS DES CAMEROUNAIS DE MAYOTTE (ACAM),rencontre des camerounais résidents ou de pass...,014040,000000,8,RTE,de Vahibé,97600,Mamoudzou,None,8 RTE de Vahibé 97600 Mamoudzou,,Koungou


In [11]:
import pandas as pd
waldec_csv = pd.read_csv('rna-nomenclature-waldec.csv', delimiter=";",
                         index_col=2).to_dict()["Libellé objet social parent"]
# pprint(waldec_csv)
waldec_csv[00000] = "AUTRES"
waldec_csv[19060] = "INTERVENTIONS SOCIALES"
waldec_csv[22010] = "REPRÉSENTATION, PROMOTION ET DÉFENSE D'INTÉRÊTS ÉCONOMIQUES"
waldec_csv[6035]  = "CULTURE, PRATIQUES D'ACTIVITÉS ARTISTIQUES, PRATIQUES CULTURELLES"
waldec_csv[40510] = "ACTIVITÉS RELIGIEUSES, SPIRITUELLES OU PHILOSOPHIQUES"
waldec_csv[17035] = "SANTÉ"

df3['social_object1_libelle'] = df3.apply(lambda row: waldec_csv[int(row["objet_social1"] or 0)], axis=1)
df3['social_object2_libelle'] = df3.apply(lambda row: waldec_csv[int(row["objet_social2"] or 0)], axis=1)
df3

,id,titre,objet,objet_social1,objet_social2,adrs_numvoie,adrs_typevoie,adrs_libvoie,adrs_codepostal,adrs_libcommune,siteweb,adrs,dept,region,social_object1_libelle,social_object2_libelle
0,W751075369,ASSOCIATION NDOG-NÉM FRANCE ANNF,créer et favoriser les contacts le dialogue dv...,014000,000000,94,RUE,des Chataigniers,95100,Argenteuil,None,94 RUE des Chataigniers 95100 Argenteuil,Val-d'Oise,Île-de-France,"AMICALES, GROUPEMENTS AFFINITAIRES, GROUPEMENT...",AUTRES
1,W751101199,CERCLE D'ETUDES DES TRADITIONS ET DE LA CULTUR...,étudier les traditions et la culture camerouna...,020015,000000,148,AV,d'Italie,75013,Paris,None,148 AV d'Italie 75013 Paris,Paris,Île-de-France,"ASSOCIATIONS CARITATIVES, HUMANITAIRES, AIDE A...",AUTRES
2,W751105303,LES AMIS DE L'INSTITUT CATHOLIQUE DE YAOUNDE C...,contribuer a la réalisation des objectifs de l...,040000,000000,21,RUE,d'Assas,75006,Paris,None,21 RUE d'Assas 75006 Paris,Paris,Île-de-France,"ACTIVITÉS RELIGIEUSES, SPIRITUELLES OU PHILOSO...",AUTRES
3,W751109411,ASSOCIATION DES CHRETIENS DU CAMEROUN POUR LE ...,"réunir, en france comme à l'étranger, au camer...",014035,000000,4,RUE,de Castellane,75008,Paris,www.accd-cameroun.org,4 RUE de Castellane 75008 Paris,Paris,Île-de-France,"AMICALES, GROUPEMENTS AFFINITAIRES, GROUPEMENT...",AUTRES
4,W751118437,ASSOCIATION CAMEROUNAISE D'ENTRAIDE DES RESSOR...,être le lieu de rencontre de tous les boh nset...,019000,000000,8,RUE,Raymond pitet,75017,Paris,None,8 RUE Raymond pitet 75017 Paris,Paris,Île-de-France,INTERVENTIONS SOCIALES,AUTRES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,W051001109,ASSOCIATION DES AMIS DU COLLEGE EVANGELIQUE DE...,soutenir le collège evangélique de bangwa (cam...,020015,000000,17,VILL1,Curial,75019,Paris,None,17 VILL1 Curial 75019 Paris,Paris,Île-de-France,"ASSOCIATIONS CARITATIVES, HUMANITAIRES, AIDE A...",AUTRES
1067,W052001700,AFRICOEUR,faciliter et promouvoir l'entraide et la solid...,009050,020020,None,CHEM1,de Pré Lacour,05140,Aspremont,None,CHEM1 de Pré Lacour 05140 Aspremont,Hautes-Alpes,Provence-Alpes-Côte d'Azur,ACTION SOCIOCULTURELLE,"ASSOCIATIONS CARITATIVES, HUMANITAIRES, AIDE A..."
1068,W2A4000401,ASSOCIATION DE SOUTIEN AUX ENFANTS CAMEROUNAIS...,association à but humanitaire. elle apporte ai...,020000,020020,None,None,Trinité,20137,Porto-Vecchio,None,Trinité 20137 Porto-Vecchio,Corse-du-Sud,Corse,"ASSOCIATIONS CARITATIVES, HUMANITAIRES, AIDE A...","ASSOCIATIONS CARITATIVES, HUMANITAIRES, AIDE A..."
1069,W9T1002080,ASSOCIATIONS DES CAMEROUNAIS DE MAYOTTE (ACAM),rencontre des camerounais résidents ou de pass...,014040,000000,8,RTE,de Vahibé,97600,Mamoudzou,None,8 RTE de Vahibé 97600 Mamoudzou,Mayotte,Mayotte,"AMICALES, GROUPEMENTS AFFINITAIRES, GROUPEMENT...",AUTRES


In [12]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="acute.mongulu.cm")

def geocode(adrs):
    return geolocator.geocode(adrs,country_codes="fr",timeout=None)

df3['geocode'] = df3.parallel_apply(lambda row: geocode(row["adrs"]), axis=1)
df3

,id,titre,objet,objet_social1,objet_social2,adrs_numvoie,adrs_typevoie,adrs_libvoie,adrs_codepostal,adrs_libcommune,siteweb,adrs,dept,region,social_object1_libelle,social_object2_libelle,geocode
0,W751075369,ASSOCIATION NDOG-NÉM FRANCE ANNF,créer et favoriser les contacts le dialogue dv...,014000,000000,94,RUE,des Chataigniers,95100,Argenteuil,None,94 RUE des Chataigniers 95100 Argenteuil,Val-d'Oise,Île-de-France,"AMICALES, GROUPEMENTS AFFINITAIRES, GROUPEMENT...",AUTRES,"(Rue des Châtaigniers, Argenteuil, Val-d'Oise,..."
1,W751101199,CERCLE D'ETUDES DES TRADITIONS ET DE LA CULTUR...,étudier les traditions et la culture camerouna...,020015,000000,148,AV,d'Italie,75013,Paris,None,148 AV d'Italie 75013 Paris,Paris,Île-de-France,"ASSOCIATIONS CARITATIVES, HUMANITAIRES, AIDE A...",AUTRES,"(148, Avenue d'Italie, Quartier de la Maison-B..."
2,W751105303,LES AMIS DE L'INSTITUT CATHOLIQUE DE YAOUNDE C...,contribuer a la réalisation des objectifs de l...,040000,000000,21,RUE,d'Assas,75006,Paris,None,21 RUE d'Assas 75006 Paris,Paris,Île-de-France,"ACTIVITÉS RELIGIEUSES, SPIRITUELLES OU PHILOSO...",AUTRES,"(21, Rue d'Assas, Quartier Notre-Dame-des-Cham..."
3,W751109411,ASSOCIATION DES CHRETIENS DU CAMEROUN POUR LE ...,"réunir, en france comme à l'étranger, au camer...",014035,000000,4,RUE,de Castellane,75008,Paris,www.accd-cameroun.org,4 RUE de Castellane 75008 Paris,Paris,Île-de-France,"AMICALES, GROUPEMENTS AFFINITAIRES, GROUPEMENT...",AUTRES,"(4, Rue de Castellane, Quartier de la Madelein..."
4,W751118437,ASSOCIATION CAMEROUNAISE D'ENTRAIDE DES RESSOR...,être le lieu de rencontre de tous les boh nset...,019000,000000,8,RUE,Raymond pitet,75017,Paris,None,8 RUE Raymond pitet 75017 Paris,Paris,Île-de-France,INTERVENTIONS SOCIALES,AUTRES,"(8, Rue Raymond Pitet, Quartier de la Plaine-d..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,W051001109,ASSOCIATION DES AMIS DU COLLEGE EVANGELIQUE DE...,soutenir le collège evangélique de bangwa (cam...,020015,000000,17,VILL1,Curial,75019,Paris,None,17 VILL1 Curial 75019 Paris,Paris,Île-de-France,"ASSOCIATIONS CARITATIVES, HUMANITAIRES, AIDE A...",AUTRES,None
1067,W052001700,AFRICOEUR,faciliter et promouvoir l'entraide et la solid...,009050,020020,None,CHEM1,de Pré Lacour,05140,Aspremont,None,CHEM1 de Pré Lacour 05140 Aspremont,Hautes-Alpes,Provence-Alpes-Côte d'Azur,ACTION SOCIOCULTURELLE,"ASSOCIATIONS CARITATIVES, HUMANITAIRES, AIDE A...",None
1068,W2A4000401,ASSOCIATION DE SOUTIEN AUX ENFANTS CAMEROUNAIS...,association à but humanitaire. elle apporte ai...,020000,020020,None,None,Trinité,20137,Porto-Vecchio,None,Trinité 20137 Porto-Vecchio,Corse-du-Sud,Corse,"ASSOCIATIONS CARITATIVES, HUMANITAIRES, AIDE A...","ASSOCIATIONS CARITATIVES, HUMANITAIRES, AIDE A...","(Vulco, T 10, Tenda, Trinité de Porto Vecchio,..."
1069,W9T1002080,ASSOCIATIONS DES CAMEROUNAIS DE MAYOTTE (ACAM),rencontre des camerounais résidents ou de pass...,014040,000000,8,RTE,de Vahibé,97600,Mamoudzou,None,8 RTE de Vahibé 97600 Mamoudzou,Mayotte,Mayotte,"AMICALES, GROUPEMENTS AFFINITAIRES, GROUPEMENT...",AUTRES,"(Route de Vahibe, Passamainty, Mamoudzou, Mayo..."


In [13]:
df3[df3.geocode.isnull()] # 176 rows n'ont pas de géocodage

,id,titre,objet,objet_social1,objet_social2,adrs_numvoie,adrs_typevoie,adrs_libvoie,adrs_codepostal,adrs_libcommune,siteweb,adrs,dept,region,social_object1_libelle,social_object2_libelle,geocode
18,W751200732,BAZOU GENERAL DE FRANCE (BGF),"promouvoir l'union , l'entraide, l'entente et ...",014045,000000,3,RES,le Gai Logis,93350,Le Bourget,None,3 RES le Gai Logis 93350 Le Bourget,Seine-Saint-Denis,Île-de-France,"AMICALES, GROUPEMENTS AFFINITAIRES, GROUPEMENT...",AUTRES,None
22,W751201224,ASSOCIATION DES ARTISTES DES MONTAGNES - ASARMON,promouvoir par enseignements et par exhibition...,006040,000000,28,RUE,de Chaufourniers,75019,Paris,None,28 RUE de Chaufourniers 75019 Paris,Paris,Île-de-France,"CULTURE, PRATIQUES D'ACTIVITÉS ARTISTIQUES, PR...",AUTRES,None
29,W751204269,(AFCAR) ASSOCIATION FRANCO-CAMEROUNAISE D'AIDE...,mise en place d'un dispositif effectif et effi...,014040,030000,50,CHEM1,de Montval à la Montagne,78160,Marly-le-Roi,None,50 CHEM1 de Montval à la Montagne 78160 Marly-...,Yvelines,Île-de-France,"AMICALES, GROUPEMENTS AFFINITAIRES, GROUPEMENT...","AIDE À L'EMPLOI, DÉVELOPPEMENT LOCAL, PROMOTIO...",None
32,W751205161,ASSOCIATION FRANCO - CAMEROUNAISE POUR LE DEVE...,filtre matriciel et guichet unique dans le por...,014045,000000,50,CHEM1,de Montval à la Montagne,78160,Marly-le-Roi,None,50 CHEM1 de Montval à la Montagne 78160 Marly-...,Yvelines,Île-de-France,"AMICALES, GROUPEMENTS AFFINITAIRES, GROUPEMENT...",AUTRES,None
49,W751216714,AMOUR & FIDELITE,réunir les ressortissants camerounais et sympa...,014040,014035,15,RUE,Saint Quentin,75010,Paris,None,15 RUE Saint Quentin 75010 Paris,Paris,Île-de-France,"AMICALES, GROUPEMENTS AFFINITAIRES, GROUPEMENT...","AMICALES, GROUPEMENTS AFFINITAIRES, GROUPEMENT...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043,W613001327,ZARRO - LE GRENIER,"accompagner humainement, socialement, techniqu...",020005,000000,11,None,L'hôtel Veron,61290,Saint-Victor-de-Réno,None,11 L'hôtel Veron 61290 Saint-Victor-de-Réno,Orne,Normandie,"ASSOCIATIONS CARITATIVES, HUMANITAIRES, AIDE A...",AUTRES,None
1045,W652000630,EXARCHAT FRANCAIS DE L'EGLISE CATHOLIQUE ET A...,aide spirituelle par therapie de l'ame pour ce...,040000,000000,None,RTE,de l'église,65250,La Barthe-de-Neste,None,RTE de l'église 65250 La Barthe-de-Neste,Hautes-Pyrénées,Occitanie,"ACTIVITÉS RELIGIEUSES, SPIRITUELLES OU PHILOSO...",AUTRES,None
1054,W191002040,ASSOCIATION CAMEROUN BRAILLE (ACB),aider les déficients visuels à surmonter leur ...,003030,000000,2,PL,de l'Église,19270,Sainte-Féréole,None,2 PL de l'Église 19270 Sainte-Féréole,Corrèze,Nouvelle-Aquitaine,"DÉFENSE DE DROITS FONDAMENTAUX, ACTIVITÉS CIVI...",AUTRES,None
1066,W051001109,ASSOCIATION DES AMIS DU COLLEGE EVANGELIQUE DE...,soutenir le collège evangélique de bangwa (cam...,020015,000000,17,VILL1,Curial,75019,Paris,None,17 VILL1 Curial 75019 Paris,Paris,Île-de-France,"ASSOCIATIONS CARITATIVES, HUMANITAIRES, AIDE A...",AUTRES,None


In [14]:
df3[df3.adrs_libcommune.isnull()] # mais aucune n'a le nom de la commune vide donc ce sera ça qui sera utilisé

,id,titre,objet,objet_social1,objet_social2,adrs_numvoie,adrs_typevoie,adrs_libvoie,adrs_codepostal,adrs_libcommune,siteweb,adrs,dept,region,social_object1_libelle,social_object2_libelle,geocode


In [15]:
from tqdm import tqdm
tqdm.pandas()
df3.loc[df3.geocode.isnull(),"geocode"] = df3[df3.geocode.isnull()].progress_apply(lambda row: geocode(row["adrs_libcommune"]), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 177/177 [00:00<00:00, 237.87it/s]


In [16]:
df3[df3.geocode.isnull()] # il reste une asso à Besancon Cedex ui est pas reconnu, on va le faire avec Besançon

,id,titre,objet,objet_social1,objet_social2,adrs_numvoie,adrs_typevoie,adrs_libvoie,adrs_codepostal,adrs_libcommune,siteweb,adrs,dept,region,social_object1_libelle,social_object2_libelle,geocode
967,W251002897,ETUDIANTS CAMEROUNAIS DE FRANCHE-COMTE,réunir les étudiants camerounais de franche-co...,015065,014040,36 A,AV,de l'Observatoire,25030,Besançon Cédex,None,36 A AV de l'Observatoire 25030 Besançon Cédex,Doubs,Bourgogne-Franche-Comté,ÉDUCATION FORMATION,"AMICALES, GROUPEMENTS AFFINITAIRES, GROUPEMENT...",None


In [17]:
df3.loc[df3.id == "W251002897","geocode"] = df3[df3.geocode.isnull()].progress_apply(lambda row: geocode("Besançon"), axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 199.64it/s]


In [18]:
df3['longitude'] = df3['geocode'].progress_apply(lambda x: x.longitude)
df3['latitude'] = df3['geocode'].progress_apply(lambda x: x.latitude)
df3 = df3.drop(columns=["objet_social1","objet_social2","geocode"])
df3.to_csv("ref-rna-real-mars-2022.csv")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1071/1071 [00:00<00:00, 394355.16it/s]


In [ ]:
from geopandas import GeoDataFrame
from shapely.geometry import Point

gdf = GeoDataFrame(
    df3.drop(['longitude', 'latitude','adrs'], axis=1),
    crs={'init': 'epsg:4326'},
    geometry=[Point(xy) for xy in zip(df3.longitude, df3.latitude)])
gdf.to_file("ref-rna-real-mars-2022.geojson", driver="GeoJSON")

In [23]:
# Gogocarto lit une liste de catégories sur un champ défini et considère la virgule comme le caractère de séparation
# On a donc opté pour remplacer la virgule(",") par le slash("/")
df3["social_object1_libelle"] = df3["social_object1_libelle"].progress_apply(lambda x: x.replace(",","/"))
df3["social_object2_libelle"] = df3["social_object2_libelle"].progress_apply(lambda x: x.replace(",","/"))
df3.to_csv("ref-rna-real-mars-2022.csv")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1071/1071 [00:00<00:00, 592781.68it/s]
